In [1]:
import nltk
import numpy as np
import pandas as pd
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sqlalchemy import create_engine
from nltk.corpus import gutenberg
import re
import spacy
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement
import warnings
warnings.filterwarnings("ignore")

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


symbolic link created for C:\ProgramData\Anaconda3\lib\site-packages\spacy\data\en <<===>> C:\ProgramData\Anaconda3\lib\site-packages\en_core_web_sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[+] Linking successful
C:\ProgramData\Anaconda3\lib\site-packages\en_core_web_sm -->
C:\ProgramData\Anaconda3\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

data = pd.read_sql_query('SELECT * FROM dialogs', con=engine)
                       
engine.dispose()
data.head()

,index,dialogs
0,0,Can we make this quick? Roxanne Korrine and A...
1,1,"Well, I thought we'd start with pronunciation,..."
2,2,Not the hacking and gagging and spitting part....
3,3,Okay... then how 'bout we try out some French ...
4,4,You're asking me out. That's so cute. What's ...


In [3]:
data.drop(columns=['index'], inplace=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304446 entries, 0 to 304445
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   dialogs  304446 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [5]:
# Nested Joins and splits to remove excess whitespace and combine into one clean text string
dialogue = ' '.join(' '.join(data).split())

In [6]:
# Use spacy to parse dialog
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
doc = nlp(dialogue)

In [7]:
# removing the stopwords while lemmatizing
nostop_lemmas = [token.lemma_ for token in doc if not token.is_stop]

In [8]:
# Instantiate a new chatbot and give it a name
chatbot = ChatBot('Movie Dialogue')
# Clear any accumulated knowledge
chatbot.storage.drop
# Create a new trainer for chatbot
trainer = ListTrainer(chatbot)
# Train the chatbot
trainer.train(nostop_lemmas)

List Trainer: [####################] 100%

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
GREETING_INPUTS = ["hello", "hi", "greetings", "what's up","hey"]
GREETING_RESPONSES = ["hello", "hi", "hey", "hi there"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [10]:
print('This is the Movie Dialogue chatbot! Type "exit" if you wish to quit chatting...')

while True:
    user_input = input('User: ')
    user_input = user_input.lower()
    
    if (user_input != 'exit'):
        if (user_input == 'thanks' or user_input == 'thank you'):
            break
            print("Moviebot: You're welcome!")
            
        else:
            if (greeting(user_input) != None):
                print("Moviebot: " + greeting(user_input))
            else:
                print("Moviebot: ", end = '')
                print(chatbot.get_response(user_input))
    else:
        print("Moviebot: Bye! It was a great chat!")
        break

This is the Movie Dialogue chatbot! Type "exit" if you wish to quit chatting...
User: hey
Moviebot: hi
User: hi
Moviebot: hello
User: hello
Moviebot: hey
User: are you a robot?
Moviebot: up
User: sounds about right
Moviebot: is
User: why do you only output one word?
Moviebot: are you a robot?
User: that is a little better
Moviebot: are you a robot?
User: no
Moviebot: sounds about right
User: what is going on?
Moviebot: seem
User: butterfly
Moviebot: dialog
User: dialog
Moviebot: this is madness
User: how did you remember that phrase?
Moviebot: dialog
User: dont crap out on me just yet
Moviebot: yo
User: i give up
Moviebot: what
User: you are not working right
Moviebot: butterfly
User: exit
Moviebot: Bye! It was a great chat!


This chatbot is extremely basic and needs much better training in order to parse out acceptable phrases. Maybe spending more time on preprocessing or using different numbers of words when building relationships is necessary to output more relevant phrases. This bot is just good at learning and outputting very basic responses as is.